In [1]:
# Q1: Some of the facilities charge a fee to members, but some do not.
# Write a SQL query to produce a list of the names of the facilities that do.

SELECT name, membercost
FROM Facilities
WHERE membercost > 0;

name             membercost
Tennis Court 1   5.0
Tennis Court 2   5.0
Massage Room 1   9.9
Massage Room 2   9.9
Squash Court     3.5

SyntaxError: invalid syntax (<ipython-input-1-e0a63aa9b7de>, line 4)

In [ ]:
# Q2: How many facilities do not charge a fee to members?

SELECT COUNT(name)
FROM Facilities
WHERE membercost = 0;

COUNT(name)
4

In [ ]:
# Q3: Write an SQL query to show a list of facilities that charge a fee to members,
# where the fee is less than 20% of the facility's monthly maintenance cost.
# Return the facid, facility name, member cost, and monthly maintenance of the
# facilities in question.

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > 0
AND membercost < (0.20 * monthlymaintenance);


facid	name	membercost	monthlymaintenance	
0	Tennis Court 1	5.0	200
1	Tennis Court 2	5.0	200
4	Massage Room 1	9.9	3000
5	Massage Room 2	9.9	3000
6	Squash Court	3.5	80

In [ ]:
#Write an SQL query to retrieve the details of facilities with ID 1 and 5.
#Try writing the query without using the OR operator.

SELECT *
FROM Facilities
WHERE facid IN (1,5);

facid	name	membercost	guestcost	initialoutlay	monthlymaintenance	
5	Massage Room 2	9.9	80.0	4000	3000
1	Tennis Court 2	5.0	25.0	8000	200

In [ ]:
#Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', 
# depending on if their monthly maintenance cost is more than $100. 
#Return the name and monthly maintenance of the facilities in question.

SELECT name, monthlymaintenance,
CASE WHEN (monthlymaintenance > 100) THEN 'expensive'
ELSE 'cheap' END AS costlabel
FROM Facilities;

name	        monthlymaintenance	costlabel	
Tennis Court 1	200	                expensive
Tennis Court 2	200	                expensive
Badminton Court	50	                cheap
Table Tennis	10	                cheap
Massage Room 1	3000	            expensive
Massage Room 2	3000	            expensive
Squash Court	80	                cheap
Snooker Table	15	                cheap
Pool Table	    15	                cheap

In [ ]:
#Q6: You'd like to get the first and last name of the last member(s) who signed up. 
#Try not to use the LIMIT clause for your solution.

SELECT firstname, surname
FROM Members
WHERE joindate = (
    SELECT MAX(joindate)
    FROm Members);

firstname surname
Darren    Smith

In [ ]:
#Produce a list of all members who have used a tennis court.
#Include in your output the name of the court, and the name of the member formatted as a single column.
#Ensure no duplicate data, and order by the member name.

SELECT DISTINCT CONCAT(m.surname, ", ",  m.firstname) AS member, f.name AS court
FROM Members AS m
INNER JOIN Bookings AS b
ON m.memid = b.memid
INNER JOIN Facilities AS f
ON f.facid = b.facid
WHERE f.name LIKE 'Tennis%'
ORDER BY member;

member	court	
Bader, Florence	Tennis Court 1
Bader, Florence	Tennis Court 2
Baker, Anne	Tennis Court 1
Baker, Anne	Tennis Court 2
Baker, Timothy	Tennis Court 2
Baker, Timothy	Tennis Court 1
Boothe, Tim	Tennis Court 1
Boothe, Tim	Tennis Court 2
Butters, Gerald	Tennis Court 2
Butters, Gerald	Tennis Court 1
Coplin, Joan	Tennis Court 1
Crumpet, Erica	Tennis Court 1
Dare, Nancy	Tennis Court 2
Dare, Nancy	Tennis Court 1
Farrell, David	Tennis Court 1
Farrell, David	Tennis Court 2
Farrell, Jemima	Tennis Court 2
Farrell, Jemima	Tennis Court 1
Genting, Matthew	Tennis Court 1
GUEST, GUEST	Tennis Court 2
GUEST, GUEST	Tennis Court 1
Hunt, John	Tennis Court 2
Hunt, John	Tennis Court 1
Jones, David	Tennis Court 1
Jones, David	Tennis Court 2
Jones, Douglas	Tennis Court 1
Joplette, Janice	Tennis Court 2
Joplette, Janice	Tennis Court 1
Owen, Charles	Tennis Court 1
Owen, Charles	Tennis Court 2

In [ ]:
#Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. 
# Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and
# the guest user's ID is always 0. Include in your output the name of the
# facility, the name of the member formatted as a single column, and the cost.
# Order by descending cost, and do not use any subqueries.

SELECT CONCAT(m.surname, ",", m.firstname) AS member, f.name AS facility,
	(CASE WHEN m.memid > 0 THEN f.membercost * b.slots 
	ELSE f.guestcost * b.slots END) AS cost
FROM Members AS m
INNER JOIN Bookings AS b
ON m.memid = b.memid
INNER JOIN Facilities AS f
ON f.facid = b.facid
WHERE starttime LIKE '2012-09-14%'
AND ((m.memid = 0 AND b.slots * f.guestcost > 30) OR (m.memid > 0 AND b.slots * f.membercost > 30))
ORDER BY cost DESC;

member	facility	cost	
GUEST,GUEST	Massage Room 2	320.0
GUEST,GUEST	Massage Room 1	160.0
GUEST,GUEST	Massage Room 1	160.0
GUEST,GUEST	Massage Room 1	160.0
GUEST,GUEST	Tennis Court 2	150.0
GUEST,GUEST	Tennis Court 1	75.0
GUEST,GUEST	Tennis Court 1	75.0
GUEST,GUEST	Tennis Court 2	75.0
GUEST,GUEST	Squash Court	70.0
Farrell,Jemima	Massage Room 1	39.6
GUEST,GUEST	Squash Court	35.0
GUEST,GUEST	Squash Court	35.0

In [ ]:
# Q9: This time, produce the same result as in Q8, but using a subquery.

SELECT member, facility, cost
FROM (
    SELECT CONCAT(m.surname, ",", m.firstname) AS member, f.name AS facility,
	(CASE WHEN m.memid > 0 THEN f.membercost * b.slots 
	ELSE f.guestcost * b.slots END) AS cost
	FROM Members AS m
	INNER JOIN Bookings AS b
	ON m.memid = b.memid
	INNER JOIN Facilities AS f
	ON f.facid = b.facid
	WHERE starttime LIKE '2012-09-14%') as reservation
WHERE cost > 30
ORDER BY cost DESC;